# Covered here

* [Syntax errors versus exceptions](#Syntax-errors-versus-exceptions)
* [Handling exceptions: `try/except`](#Handling-exceptions:-try/except)
* [Assert](#Assert)

# Raising exceptions

In principle, **an exception is just an object**, like everything else in the language.  The one thing all exceptions have in common is that they inherit from a built-in class called `BaseException`.

In [1]:
issubclass(Exception, BaseException)

True

# Syntax errors versus exceptions

There are (at least) two distinguishable kinds of errors: _syntax errors_ and _exceptions_.  (Although a `SyntaxError` is technically just a type of exception.)

In [2]:
issubclass(SyntaxError, Exception)

True

## Syntax errors

Syntax errors (parsing errors) - the most common kind of everyday complaint.  The parser repeats the offending line and displays a little ‘arrow’ pointing at the earliest point in the line where the error was detected.

In [1]:
while True print('Hello world')

SyntaxError: invalid syntax (<ipython-input-1-614901b0e5ee>, line 1)

## Exceptions

**Even if a statement or expression is syntactically correct**, it may cause an error when an attempt is made to execute it. **Errors detected during execution are called _exceptions_** and are not unconditionally fatal: you will soon learn how to handle them in Python programs. 

Most exceptions are not handled by programs, however, and result in different error messages:

In [1]:
1/0

ZeroDivisionError: division by zero

In [2]:
d.sum()

NameError: name 'd' is not defined

The last line of the error message indicates what happened.  The string printed as the exception type is the name of the built-in exception that occurred.  Built-in exceptions are listed [here](https://docs.python.org/3/library/exceptions.html#bltin-exceptions).

There are many different exception classes available, but we can easily define more of our own.  [Here](https://github.com/pydata/pandas-datareader/blob/master/pandas_datareader/_utils.py#L14) is one such example from `pandas-datareader`.

Below, two objects are newly constructed from the built-in classes `TypeError` and `ValueError`:

In [3]:
class EvenOnly(list): # class inheritance
    def append(self, integer):
        if not isinstance(integer, int):
            raise TypeError("Only integers can be added")
        if integer % 2:
            raise ValueError("Only even numbers can be added")
        super().append(integer)
e = EvenOnly()
e.append(2); e

[2]

When an exception is raised, it appears to stop program execution immediately.  Any lines that were supposed to run after the exception is raised are not executed, and unless the exception is dealt with, the program will exit with an error message.

In [4]:
def no_return():
    print("I am about to raise an exception")
    raise Exception("This is always raised")
    print("This line will never execute")
    return "I won't be returned"
no_return()

I am about to raise an exception


Exception: This is always raised

Furthermore, if we have a function that calls another function that raises an exception, nothing will be executed in the first function after the point where the second function was called:

In [5]:
def call_exceptor():
    print("call_exceptor starts here...")
    no_return()
    print("an exception was raised...")
    print("...so these lines don't run")
call_exceptor()    

call_exceptor starts here...
I am about to raise an exception


Exception: This is always raised

The class below extends the list built-in and overrides the `append` method to check two conditions that ensure the item is an even integer:

In [1]:
class EvenOnly(list):
    def append(self, integer):
        if not isinstance(integer, int):
            raise TypeError("Only integers can be added")
        if integer % 2:
            raise ValueError("Only even numbers can be added")
        super().append(integer)

# Handling exceptions: `try/except`

* Errors can be handled with `try` and `except` statements.
* The `try/except` structure is **meant to deal with the fact that an exception would otherwise stop the program in its tracks when the exception is raised.**
* `Try` statement: The code that could potentially have an error is put in a `try` clause.
* `Except`: The program execution moves to the start of a following `except` clause if an error happens.

In [3]:
def fn_op(divideBy):
    try:
        return 42/divideBy
    except:                 # naked except statements are generally discouraged
        print('Error')
fn_op(0)

Error


Continuing with the above example of `no_return()`, using `try/except` allows us to create a function that can "recover and continue" after an exception:

In [7]:
def yes_return():
    try:
        no_return()
    except:
        print("I caught an exception")
    print("executed after the exception")
yes_return()    

I am about to raise an exception
I caught an exception
executed after the exception


The problem with the preceding code is that it will catch any type of exception.  Consider the following instead:

In [8]:
def fn_op2(divideBy):
    try:
        return 42/divideBy
    except ZeroDivisionError:
        print('A specific type of error')
fn_op2('the')

TypeError: unsupported operand type(s) for /: 'int' and 'str'

In [9]:
fn_op2(0)

A specific type of error


Google recommends against using the "catch-all" `except` statement.  Python is very tolerant in this regard and `except:` will really catch everything including misspelled names, `sys.exit()` calls, Ctrl+C interrupts, unittest failures and all kinds of other exceptions that you simply don't want to catch.

We can even **catch two or more different exceptions and handle them with the same code**. Here's an example that raises three different types of exception. It handles `TypeError` and `ZeroDivisionError` with the same exception handler, but it may also raise a `ValueError` if you supply the number 13: 

In [10]:
def funny_division2(anumber):
    try:
        if anumber == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / anumber
    except (ZeroDivisionError, TypeError):
        return "Enter a number other than zero"

for val in (0, "hello", 50.0, 13):
    print("Testing {}:".format(val), end=" ")
    print(funny_division2(val))

Testing 0: Enter a number other than zero
Testing hello: Enter a number other than zero
Testing 50.0: 2.0
Testing 13: 

ValueError: 13 is an unlucky number

## Returning the object

Sometimes, when we catch an exception, we need a reference to the `Exception` object itself. This most often happens when we define our own exceptions with custom arguments, but can also be relevant with standard exceptions.  The snippet below prints out the string argument that we passed into `ValueError` upon initialization.

In [2]:
try:
    raise ValueError("This is an argument")
except ValueError as e:
    print("The exception arguments were", e.args)

The exception arguments were ('This is an argument',)


# Assert

* Python’s built-in `assert` statement is a debugging aid that tests a condition.
 * If the condition is true, it does nothing and your program just continues to execute. 
 * If the assert condition evaluates to false, it raises an `AssertionError` exception (optional error message).
* The proper use of assertions is to inform developers about unrecoverable errors in a program. Assertions are not intended to signal expected error conditions, like a `File-Not-Found` error, where a user can take corrective action or just try again.
* Assertions are meant to be internal self-checks for your program. They work by declaring some conditions as impossible in your code.

In [8]:
def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price'], 'Message'
    return price
shoes = {'name': 'Fancy Shoes', 'price': 14900}
apply_discount(shoes, 0.25)

11175

In [9]:
apply_discount(shoes, 2.0)

AssertionError: Message

Why not just use a `try` & `except` or `if-else` exception?

The proper use of assertions is to inform developers about unrecoverable errors in a program. **Assertions are not intended to signal expected error conditions where a user can take corrective action or just try again.**